In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

loading the data

In [ ]:
data = pd.read_csv("/kaggle/input/wisconsin-breast-cancer-cytology-features/wisconsin_breast_cancer.csv")
data.head()

In [ ]:
data.info()

removing null values / Replacing them.

In [ ]:
new_data = data.dropna()
new_data.info()

splitting the features into X and y

In [ ]:
y = new_data.iloc[:,10:11]
X = new_data.iloc[:,1:10]
print(y)
print(X)

splitting the X and y into test and train using stratify on y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,stratify=y,random_state=1502775)
X_train.head()
X_train.info()

In [ ]:
from sklearn.linear_model import SGDClassifier


function for getting all subsets

In [ ]:
def powerset(set):
    if set == []:
        yield set
    else:
        for i in powerset(set[1:]):
            yield i
            yield [set[0]] + i


using the SGDClassifier to get the cross validation Scores

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

cv_scores = list()
cv_pred_accu = list()
best_set = list()
for i in powerset(list(X_train.columns)):
    if(i!=[]):
        sgd_clf = SGDClassifier(random_state=1502775)
        cross_validate(sgd_clf, X_train[i], y_train.values.ravel(), cv = 10)
        results = cross_val_score(sgd_clf, X_train[i], y_train.values.ravel(), cv = 10)
        cv_scores.append(results.mean())
        cv_pred_accu.append(accuracy_score(y_test, cross_val_predict(sgd_clf, X_test, y_test.values.ravel(), cv = 10)))
        best_set.append(i)
    

lets print the max cross validation accuracy along with its prediction score and the subset it corrisponds to 

In [ ]:
print(max(cv_scores))
print(cv_pred_accu[cv_scores.index(max(cv_scores))])  
cv_best=best_set[cv_scores.index(max(cv_scores))]
print(cv_best)  

the max cv accuracy is 97.4% which is pretty good but it has a prediction score of 94% which could be better

fitting the train data of each subset and predicting

In [ ]:
test_score=list()
subset = list()
for i in powerset(list(X_train.columns)):
    if(i!=[]):
        sgd_clf = SGDClassifier(random_state=1502775)
        sgd_clf.fit(X_train[i],y_train.values.ravel())
        test_score.append(accuracy_score(y_test,sgd_clf.predict(X_test[i])))
        subset.append(i)

In [ ]:
print(max(test_score))
print(subset[test_score.index(max(test_score))])
print()
print("test score for subset with best cv_accuracy:")
print(test_score[subset.index(cv_best)])

Which subset gives the best cross-validation scores?
     : ['thickness', 'size', 'adhesion', 'single', 'nuclei', 'chromatin']
       with cross-validation score = 0.9761952861952862
       and  prediction-accuracy = 0.9635036496350365
       
What is the accuracy of this subset on the test data?
    : 0.9635036496350365
    
Is that the best possible test data accuracy?
       : No, the best test data accuracy is given by the subset :
        ['size', 'adhesion', 'nuclei', 'chromatin', 'nucleoli']
        accuracy : 0.9781021897810219

scatter plot

In [ ]:
import matplotlib.pyplot as plt
plt.xlabel("cv-accuracy")
plt.ylabel("test-accuracy")
plt.scatter(cv_scores,test_score,alpha=0.4)
plt.show()

DOING THE SAME WITH A RANDOMFORESTCLASSIFIER

In [ ]:
from sklearn.ensemble import RandomForestClassifier
all_scoresRF = list()
pred_accuRF = list()
best_setRF = list()
for i in powerset(list(X_train.columns)):
    if(i!=[]):
        rf_clf = RandomForestClassifier(n_estimators=30,random_state=1502775)
        cross_validate(rf_clf, X_train[i], y_train.values.ravel(), cv = 10)
        results = cross_val_score(rf_clf, X_train[i], y_train.values.ravel(), cv = 10)
        all_scoresRF.append(results.mean())
        pred_accuRF.append(accuracy_score(y_test, cross_val_predict(rf_clf, X_test, y_test.values.ravel(), cv = 10)))
        best_setRF.append(i)

In [ ]:
print(max(all_scoresRF))
print(pred_accuRF[all_scoresRF.index(max(all_scoresRF))])  
cv_bestRF=best_setRF[all_scoresRF.index(max(all_scoresRF))]
print(cv_bestRF)

the max cv accuracy is 97.8% which is almost the same as the SGDClassifier but it has a prediction score of 96.4% which is much better.

In [ ]:
test_scoreRF=list()
subsetRF = list()
for i in powerset(list(X_train.columns)):
    if(i!=[]):
        rf_clf = RandomForestClassifier(n_estimators=30,random_state=1502775)
        rf_clf.fit(X_train[i],y_train.values.ravel())
        test_scoreRF.append(accuracy_score(y_test,rf_clf.predict(X_test[i])))
        subsetRF.append(i)

In [ ]:
print("Prediction Accuracy")
print(max(test_scoreRF))
print()
print("Subset with best prediction accuracy")
print(subsetRF[test_scoreRF.index(max(test_scoreRF))])
print()
print("test score for subset with best cv_accuracy:")
print(test_scoreRF[subsetRF.index(cv_bestRF)])

Which subset gives the best cross-validation scores? : 
['thickness', 'size', 'shape', 'adhesion', 'single', 'nuclei', 'nucleoli']with 
 cross-validation score = 0.978013468013468 
 prediction-accuracy = 0.9635036496350365

What is the accuracy of this subset on the test data? : 
0.9708029197080292

Is that the best possible test data accuracy? :
No, the best test data accuracy is given by the subset : 
['thickness', 'shape', 'single', 'nuclei', 'chromatin', 'nucleoli']
accuracy : 0.9781021897810219

In [ ]:
plt.xlabel("cv-accuracy")
plt.ylabel("test-accuracy")
plt.scatter(all_scoresRF,test_scoreRF,alpha=0.4)
plt.show()